# Import Libraries

In [1]:
import os
import cv2
import random
import numpy as np
from imutils import paths
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,RMSprop
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import csv

Using TensorFlow backend.


# Reading Input Data

In [2]:
#get all the input data paths and randomly shuffle them
data_paths = sorted(list(paths.list_images("../input/hestia-epoch/train data/Train data")))

random.seed(1203)
random.shuffle(data_paths)

data = []
labels = []
encode = {"leukocoria":1,"non-leukocoria":0}
#add img array to data
for path in data_paths:
    #read image from image path
    img = cv2.imread(path)
    img = cv2.GaussianBlur(img,(5,5),0)
    img = cv2.resize(img,(28,28))
    img = img_to_array(img)

    #add images to data list
    data.append(img)
    
    #add encoded lable to label list
    label = path.split(os.path.sep)[-2]
    labels.append(encode[label])
    
    

# Data Preprocessing

In [3]:
#scaling data
data = np.array(data, dtype = float) / 255.0
labels = np.array(labels)

#train-test split
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = 0, random_state = 1203)

#one hot encoding of labels
trainY = to_categorical(trainY, num_classes = 2)
testY = to_categorical(testY, num_classes = 2)

#data augmentation for image generator
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True)


# The Model -1

In [4]:

#some parameters for the model
height = 28
width = 28
depth = 3
classes = 2

model = Sequential()
input_shape = (height, width, depth)
model.add(Conv2D(20,(5,5), padding = "same", input_shape = input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
model.add(Conv2D(50, (5,5), padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

#softmax
model.add(Dense(classes))
model.add(Activation("softmax"))


Instructions for updating:
Colocations handled automatically by placer.


# The Model -2

# Training

In [5]:
from math import ceil
EPOCHS = 60
INIT_LR = 1e-3
BS = 32

print("[Compiling the Model]")
opt = Adam(lr = INIT_LR, decay=INIT_LR/EPOCHS)
#opt = RMSprop(lr = INIT_LR, decay=INIT_LR/EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics = ["accuracy"])

print("[Training]")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size = BS), steps_per_epoch = ceil(len(trainX) / BS), epochs = EPOCHS, verbose = 1)


[Compiling the Model]
[Training]
Instructions for updating:
Use tf.cast instead.
Epoch 1/60
28/28 [==============================] - 3s 90ms/step - loss: 0.5537 - acc: 0.7733
Epoch 2/60
28/28 [==============================] - 2s 68ms/step - loss: 0.4748 - acc: 0.7811
Epoch 3/60
28/28 [==============================] - 2s 68ms/step - loss: 0.4104 - acc: 0.8068
Epoch 4/60
28/28 [==============================] - 2s 68ms/step - loss: 0.3752 - acc: 0.8509
Epoch 5/60
28/28 [==============================] - 2s 68ms/step - loss: 0.3570 - acc: 0.8554
Epoch 6/60
28/28 [==============================] - 2s 68ms/step - loss: 0.3385 - acc: 0.8677
Epoch 7/60
28/28 [==============================] - 2s 68ms/step - loss: 0.2933 - acc: 0.8721
Epoch 8/60
28/28 [==============================] - 2s 68ms/step - loss: 0.2779 - acc: 0.8844
Epoch 9/60
28/28 [==============================] - 2s 68ms/step - loss: 0.2921 - acc: 0.8889
Epoch 10/60
28/28 [==============================] - 2s 67ms/step - loss:

# Evaluating The Model

In [6]:
data_paths = sorted(list(paths.list_images("../input/hestia-epoch/evaluation data/Evaluation data")))
a = 0.3
b = 0.7

#variable for writing to the csv file
with open("submission2.csv",'a') as f:
    writer = csv.writer(f)
    writer.writerow(["Id","Category"])
    for path in data_paths:
        img = cv2.imread(path)
        img = cv2.GaussianBlur(img,(5,5),0)
        img = cv2.resize(img,(28,28))
        img = img.astype("float")/255.0
        img = img_to_array(img)
        img = np.expand_dims(img, axis = 0)

        (neg, pos) = model.predict(img)[0]
        neg = (neg + a)/2
        pos = (pos + b)/2
        label = 0 if neg > pos else 1
        i = path.split(os.path.sep)[-1]
        i = i[:-4]
        
        row = [i, label]
        print(row)
        #print(i,neg, pos)
        writer.writerow(row)
        
f.close()
        
        
    
    
    

['0', 0]
['1', 0]
['10', 0]
['100', 0]
['101', 1]
['102', 0]
['103', 0]
['104', 0]
['105', 0]
['106', 0]
['107', 0]
['108', 0]
['109', 0]
['11', 0]
['110', 0]
['111', 0]
['112', 0]
['113', 0]
['114', 0]
['115', 1]
['116', 0]
['117', 1]
['118', 0]
['119', 0]
['12', 0]
['120', 0]
['121', 1]
['122', 0]
['123', 0]
['124', 0]
['125', 1]
['126', 0]
['127', 0]
['128', 0]
['129', 0]
['13', 0]
['130', 0]
['131', 0]
['132', 1]
['133', 0]
['134', 0]
['135', 0]
['136', 0]
['137', 0]
['138', 0]
['139', 0]
['14', 1]
['140', 1]
['141', 0]
['142', 0]
['143', 0]
['144', 0]
['145', 1]
['146', 0]
['147', 1]
['148', 0]
['149', 0]
['15', 0]
['150', 0]
['151', 0]
['152', 0]
['153', 1]
['154', 1]
['155', 0]
['156', 0]
['157', 0]
['158', 1]
['159', 0]
['16', 0]
['160', 0]
['161', 0]
['162', 0]
['163', 0]
['164', 0]
['165', 1]
['166', 1]
['167', 0]
['168', 0]
['169', 0]
['17', 0]
['170', 0]
['171', 0]
['172', 0]
['173', 0]
['174', 0]
['175', 1]
['176', 0]
['177', 0]
['178', 0]
['179', 0]
['18', 1]
['180', 0]
[